In [23]:
import os
# API Private Key
os.environ['OPENAI_API_KEY'] = ""

In [25]:
import openai

def get_response(input_text):
    prompt = f"請根據以下敘述寫一首詩詞: {input_text}"
    response = openai.chat.completions.create(
    model="gpt-4-1106-preview",
    # model = "gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return response


In [26]:
def get_price(response):
    price = response.usage.prompt_tokens/1000 * 0.01 + \
            response.usage.completion_tokens/1000 * 0.03 
    return price

In [27]:
import pandas as pd
import json
from tqdm import tqdm

def process_json(input_file, output_file, num_examples=1000, chunk_size=1):
    df = pd.read_json(input_file)
    if num_examples is None:
        num_examples = df.shape[0]
    
    data = []
    cost = 0
    
    progress_bar = tqdm(total=num_examples, desc="Processing CSV", unit="row")

    for idx, row in df.head(num_examples).iterrows():
        url = row['url']
        paragraph = row['paragraph']
        response = get_response(paragraph)
        answer = response.choices[0].message.content
        cost += get_price(response)

        example_data = {
            'url': url,
            'paragraph': paragraph,
            'answer': answer
        }
        data.append(example_data)
        
        # Update the progress bar description and refresh the display
        progress_bar.set_description(f"Processing CSV - Cost: {cost:.2f}")
        progress_bar.update(1)

        if (idx + 1) % chunk_size == 0 or idx == df.shape[0] - 1:
            # Write to JSON file every chunk_size examples or on the last example
            with open(output_file, 'w', encoding='utf-8') as jsonfile:
                json.dump(data, jsonfile, ensure_ascii=False, indent=2)
    
    # Close the progress bar
    progress_bar.close()


In [28]:

input_file = './112-1_ADL_Final/InstructBLIP/InstructBlip_ver2.json'
output_file = 'output.json'
process_json(input_file, output_file)

Processing CSV - Cost: 5.23: 100%|██████████| 1000/1000 [3:24:51<00:00, 12.29s/row]
